In [48]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

In [49]:
# DataFrame des vols
flight_data = {
    'departure_city': ['Paris', 'Paris', 'Nice', 'Nice'],
    'destination_city': ['Nice', 'Nice', 'Paris', 'Paris'],
    'start_date': ['2024-06-01', '2024-06-01', '2024-06-01', '2024-06-01'],
    'duration': [2, 1.5, 1.75, 2],
    'stopovers': [0, 1, 0, 0],
    'price': [100, 150, 120, 130],
}
flight_df = pd.DataFrame(flight_data)

# DataFrame des hôtels
hotel_data = {
    'city': ['Nice', 'Lyon', 'Marseille'],
    'check_in_date': ['2024-06-01', '2024-06-01', '2024-06-01'],
    'price_per_night': [60, 50, 55],
    'rating': [4.5, 4.0, 4.2]
}
hotel_df = pd.DataFrame(hotel_data)

# DataFrame des villes avec les thèmes
city_data = {
    'city': ['Nice', 'Lyon', 'Marseille'],
    'beach': [True, False, True],
    'family': [True, True, True],
    'ski': [False, False, False],
    'golf': [True, False, True]
}
city_df = pd.DataFrame(city_data)


In [50]:
flight_df.head()

,departure_city,destination_city,start_date,duration,stopovers,price
0,Paris,Nice,2024-06-01,2.00,0,100
1,Paris,Nice,2024-06-01,1.50,1,150
2,Nice,Paris,2024-06-01,1.75,0,120
3,Nice,Paris,2024-06-01,2.00,0,130


In [51]:
hotel_df.head()

,city,check_in_date,price_per_night,rating
0,Nice,2024-06-01,60,4.5
1,Lyon,2024-06-01,50,4.0
2,Marseille,2024-06-01,55,4.2


In [52]:
city_df.head()

,city,beach,family,ski,golf
0,Nice,True,True,False,True
1,Lyon,False,True,False,False
2,Marseille,True,True,False,True


In [53]:
# Modèle pour les vols
# Pré-traitement des données
flight_df['start_date'] = pd.to_datetime(flight_df['start_date'])
flight_df['start_day'] = flight_df['start_date'].dt.day
flight_df['start_month'] = flight_df['start_date'].dt.month
flight_df['start_year'] = flight_df['start_date'].dt.year

flight_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['duration', 'start_day', 'start_month', 'start_year']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['departure_city', 'destination_city', 'stopovers'])
    ])

# Pipeline de modélisation pour les vols
flight_model = Pipeline(steps=[
    ('preprocessor', flight_preprocessor),
    ('regressor', RandomForestRegressor())
])

# Split des données
X_flight = flight_df.drop(columns=['price', 'start_date'])
y_flight = flight_df['price']
X_flight_train, X_flight_test, y_flight_train, y_flight_test = train_test_split(X_flight, y_flight, test_size=0.2, random_state=42)

# Entraînement du modèle
flight_model.fit(X_flight_train, y_flight_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['duration', 'start_day',
                                                   'start_month',
                                                   'start_year']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['departure_city',
                                                   'destination_city',
                                                   'stopovers'])])),
                ('regressor', RandomForestRegressor())])

In [54]:
# Modèle pour les hôtels
# Pré-traitement des données
hotel_df['check_in_date'] = pd.to_datetime(hotel_df['check_in_date'])
hotel_df['check_in_day'] = hotel_df['check_in_date'].dt.day
hotel_df['check_in_month'] = hotel_df['check_in_date'].dt.month
hotel_df['check_in_year'] = hotel_df['check_in_date'].dt.year

hotel_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['check_in_day', 'check_in_month', 'check_in_year']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['city'])
    ])

# Pipeline de modélisation pour les hôtels
hotel_model = Pipeline(steps=[
    ('preprocessor', hotel_preprocessor),
    ('regressor', RandomForestRegressor())
])

# Split des données
X_hotel = hotel_df.drop(columns=['price_per_night', 'check_in_date', 'rating'])
y_hotel = hotel_df['price_per_night']
X_hotel_train, X_hotel_test, y_hotel_train, y_hotel_test = train_test_split(X_hotel, y_hotel, test_size=0.2, random_state=42)

# Entraînement du modèle
hotel_model.fit(X_hotel_train, y_hotel_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['check_in_day',
                                                   'check_in_month',
                                                   'check_in_year']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['city'])])),
                ('regressor', RandomForestRegressor())])

In [55]:
def predict_trip_cost(departure_city, destination_city, start_date, end_date, flight_model, hotel_model):
    """Prédire le coût d'un voyage en fonction des villes de départ et d'arrivée, des dates de départ et de retour, et des modèles de vol et d'hôtel.

    Args:
        departure_city (str): Ville de départ du voyage.
        destination_city (str): Ville d'arrivée du voyage.
        start_date (str): Date de départ du voyage.
        end_date (str): Date de retour du voyage.
        flight_model: Modèle de prédiction des prix des vols.
        hotel_model: Modèle de prédiction des prix des hôtels.

    Returns:
        dict: Dictionnaire contenant les prix des vols aller-retour, le prix total de l'hôtel et le coût total du voyage.
    """
    # Préparer les données pour la prédiction des vols
    start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

    flight_outbound = {
        'departure_city': [departure_city],
        'destination_city': [destination_city],
        'duration': [2],  # Exemple de durée de vol, vous pouvez ajuster selon les données réelles
        'stopovers': [0],  # Exemple sans escale
        'start_day': [start_date_dt.day],
        'start_month': [start_date_dt.month],
        'start_year': [start_date_dt.year]
    }
    flight_inbound = {
        'departure_city': [destination_city],
        'destination_city': [departure_city],
        'duration': [2],  # Exemple de durée de vol, vous pouvez ajuster selon les données réelles
        'stopovers': [0],  # Exemple sans escale
        'start_day': [end_date_dt.day],
        'start_month': [end_date_dt.month],
        'start_year': [end_date_dt.year]
    }

    flight_outbound_df = pd.DataFrame(flight_outbound)
    flight_inbound_df = pd.DataFrame(flight_inbound)

    outbound_price = flight_model.predict(flight_outbound_df)[0]
    inbound_price = flight_model.predict(flight_inbound_df)[0]

    # Préparer les données pour la prédiction des hôtels
    duration_days = (end_date_dt - start_date_dt).days

    hotel = {
        'city': [destination_city],
        'check_in_day': [start_date_dt.day],
        'check_in_month': [start_date_dt.month],
        'check_in_year': [start_date_dt.year]
    }

    hotel_df = pd.DataFrame(hotel)

    hotel_price_per_night = hotel_model.predict(hotel_df)[0]
    total_hotel_price = hotel_price_per_night * duration_days

    # Calculer le coût total du séjour
    total_cost = outbound_price + inbound_price + total_hotel_price

    return {
        'outbound_flight_price': outbound_price,
        'inbound_flight_price': inbound_price,
        'total_hotel_price': total_hotel_price,
        'total_cost': total_cost
    }

# Exemple d'utilisation
departure_city = 'Paris'
destination_city = 'Nice'
start_date = '2024-06-01'
end_date = '2024-06-10'

trip_cost = predict_trip_cost(departure_city, destination_city, start_date, end_date, flight_model, hotel_model)
print(trip_cost)

{'outbound_flight_price': 109.3, 'inbound_flight_price': 125.8, 'total_hotel_price': 470.7, 'total_cost': 705.8}


In [56]:
def is_trip_valid(destination_city, trip_cost, budget, themes, city_df):
    """Vérifier si un voyage est valide en fonction de la ville de destination, du coût du voyage, du budget, des thèmes et des informations sur la ville.

    Args:
        destination_city (str): Ville de destination du voyage.
        trip_cost (dict): Coût du voyage avec les détails des prix des vols et de l'hôtel.
        budget (float): Budget maximal pour le voyage.
        themes (list[str]): Liste des thèmes souhaités pour le voyage.
        city_df: DataFrame contenant les informations sur les villes, y compris les thèmes.

    Returns:
        bool: True si le voyage est valide, False sinon.
    """
    # Vérifier le budget
    total_cost = trip_cost['total_cost']

    if total_cost > budget:
        return False

    # Vérifier les thèmes
    city_info = city_df[city_df['city'] == destination_city].iloc[0]
    city_themes = {
        'beach': city_info['beach'],
        'family': city_info['family'],
        'ski': city_info['ski'],
        'golf': city_info['golf']
    }

    for theme in themes:
        if not city_themes.get(theme, False):
            return False

    return True

In [57]:
def find_possible_destinations(departure_city, start_date, end_date, budget, themes, flight_model, hotel_model, city_df):
    """Trouver les destinations possibles en fonction de la ville de départ, des dates de départ et de retour, du budget, des thèmes, des modèles de vol et d'hôtel, et des informations sur les villes.

    Args:
        departure_city (str): ville de départ du voyage
        start_date (str): date de départ du voyage
        end_date (str): date de retour du voyage
        budget (float): budget total pour le voyage
        themes (list[str]): liste des thèmes souhaités pour la destination
        flight_model: modèle de prédiction des prix des vols
        hotel_model: modèle de prédiction des prix des hôtels
        city_df: DataFrame contenant les informations sur les villes

    Returns:
        list[dict]: liste des destinations possibles avec les détails du voyage
    """
    possible_destinations = []
    for destination_city in city_df['city']:
        trip_cost = predict_trip_cost(departure_city, destination_city, start_date, end_date, flight_model, hotel_model)
        if is_trip_valid(destination_city, trip_cost, budget, themes, city_df):
            data = {
                'departure_city': departure_city,
                'destination_city': destination_city,
                'start_date': start_date,
                'end_date': end_date,
                'budget': budget,
                'trip_cost': trip_cost
            }
            possible_destinations.append(data)
    return possible_destinations

In [58]:
# Exemple d'utilisation
departure_city = 'Paris'
start_date = '2024-06-01'
end_date = '2024-06-10'
budget = 1000
themes = ['beach', 'family']

possible_destinations = find_possible_destinations(departure_city, start_date, end_date, budget, themes, flight_model, hotel_model, city_df)
print(possible_destinations)

[{'departure_city': 'Paris', 'destination_city': 'Nice', 'start_date': '2024-06-01', 'end_date': '2024-06-10', 'budget': 1000, 'trip_cost': {'outbound_flight_price': 109.3, 'inbound_flight_price': 125.8, 'total_hotel_price': 470.7, 'total_cost': 705.8}}, {'departure_city': 'Paris', 'destination_city': 'Marseille', 'start_date': '2024-06-01', 'end_date': '2024-06-10', 'budget': 1000, 'trip_cost': {'outbound_flight_price': 114.3, 'inbound_flight_price': 122.6, 'total_hotel_price': 481.95, 'total_cost': 718.8499999999999}}]
